- soft dtw divergence : https://rtavenar.github.io/ml4ts_ensai/contents/align/softdtw.html
- contraindre sur la digonale
- comaparer L2 et DTW
- Autoencoder
- VAE
- Detection d'anomalies

In [9]:
import torch
from dtw_soft import soft_dtw, soft_min
from tslearn import metrics

# test soft_min

In [10]:
# create list of values
list_a = [9, 10, 0.5, 1, 2, 3, 4, 100, -1]
gamma = 1

# calculate softmin
_min = soft_min(list_a, gamma)
print(_min)

# calculate softmin with gamma = 0
_min = soft_min(list_a, 0)
print(_min)

tensor(-1.3600)
tensor(-1.)


# our code

In [14]:
x = torch.Tensor([-2, 1, 1])
y = torch.Tensor([1, 3, 1, 9, 1])

In [18]:
# calculate soft-DTW distance
gamma = 0.5
dist, R = soft_dtw(x, y, gamma=gamma)
print(dist)

tensor(75.7575)


# Library code

In [17]:
metrics.soft_dtw(x, y, gamma=gamma)

tensor(75.7575, dtype=torch.float64)